In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

"Litigation risk is most often determined using a logistic model developed by Kim and Skinner (2012), which adds six explanatory variables (total assets, sales growth, return, return skewness, return standard deviation, and turnover) to the industry classification approach (i.e., an indicator variable for firms in the biotechnology, computers, electronics, and retail industries) developed by Francis et al. (1994)."

* Total Assets
* Sales Growth
* Stock Return Rates
    * Market-adjusted 12-month stock return. For sued firms, the accumulation period ends with the lawsuit class period end month. For non sued firms, the accumulation period ends with the fiscal year-end month
    * Market/risk asjusted cacl - https://www.investopedia.com/terms/r/riskadjustedreturn.asp
        * Sharpe Ratio
* Stock Return Skewness
    * Skewness of the firm’s 12-month return
    * Calc - https://corporatefinanceinstitute.com/resources/knowledge/other/skewness/
        * Pearson's Median Skew
* Stock Return Std Dev
    * Standard deviation of the firm’s 12-month returns
* Share Turnover
    * Trading volume accumulated over the 12-month period ending with
the lawsuit class period end month (for sued firms), and the fiscal
year-end month (for non sued firms) scaled by beginning of the year
shares outstanding. Note that the coefficient on TURNOVER is
multiplied by 1000 for expositional convenience

In [2]:
!ls

'Calculating Stock Return Workbook.xlsx'
 Russ-tickers.csv
 Russ-tickers.txt
 Untitled.ipynb
'WRDS output - Russell companies financials-data_date.csv'
'WRDS output - Russell companies financials-fiscal_year.csv'
'WRDS output - Russell companies info.csv'
'WRDS output - Russell companies stock returns-and_ratios.csv'
'WRDS output - Russell companies stock returns.csv'
'~$Calculating Stock Return Workbook.xlsx'


Financials:

In [97]:
dff = pd.read_csv('WRDS output - Russell companies financials-data_date.csv')
dff1 = dff.copy()
dff1['fyear'] = dff1['fyear'] + 1
dff1 = dff1.rename(columns = {'sale': 'sale-1'})
print(dff.shape)
dff = dff.merge(dff1[['sale-1', 'fyear', 'tic']], left_on = ['fyear', 'tic'], right_on = ['fyear', 'tic'])
print(dff.shape)

(27881, 19)
(25078, 20)


In [98]:
dff['sales_growth'] = (dff['sale'] - dff['sale-1']) / dff['sale-1']
dff['log_assets'] = np.log(dff['at'])
dff['yr_mo'] = dff['datadate'].astype('str').str[:6]
dff

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,cik,costat,ggroup,gind,gsector,gsubind,sale-1,sales_growth,log_assets,yr_mo
0,1004,20110531,2010.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1750.0,A,2010,201010,20,20101010,1352.151,0.313302,7.440573,201105
1,1004,20120531,2011.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1750.0,A,2010,201010,20,20101010,1775.782,0.168217,7.694235,201205
2,1004,20130531,2012.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1750.0,A,2010,201010,20,20101010,2074.498,0.044638,7.667111,201305
3,1004,20140531,2013.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1750.0,A,2010,201010,20,20101010,2167.100,-0.060957,7.695985,201405
4,1004,20150531,2014.0,INDL,C,D,STD,AIR,AAR CORP,USD,...,1750.0,A,2010,201010,20,20101010,2035.000,-0.216560,7.323171,201505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25073,328795,20181231,2018.0,INDL,C,D,STD,ACA,ARCOSA INC,USD,...,1739445.0,A,2010,201030,20,20103010,1462.400,-0.001368,7.683496,201812
25074,328795,20191231,2019.0,INDL,C,D,STD,ACA,ARCOSA INC,USD,...,1739445.0,A,2010,201030,20,20103010,1460.400,0.189332,7.741751,201912
25075,328795,20201231,2020.0,INDL,C,D,STD,ACA,ARCOSA INC,USD,...,1739445.0,A,2010,201030,20,20103010,1736.900,0.114399,7.881069,202012
25076,345920,20191231,2019.0,INDL,C,D,STD,HYFM,HYDROFARM HLDNG GP INC,USD,...,1695295.0,A,2010,201060,20,20106015,211.813,0.109993,5.223869,201912


Returns:

In [99]:
dfr = pd.read_csv('WRDS output - Russell companies stock returns-and_ratios.csv')
dfr['DIVAMT'] = dfr['DIVAMT'].fillna(0) # dividend amounts to 0 instead of na
print(dfr.shape)
dfr = dfr.dropna() # dropping null values
print(dfr.shape)
dfr['yr_mo'] = dfr['date'].astype('str').str[:6]
dfr['log_turnover'] = np.log(dfr['Share_Turnover'])

(309548, 13)
(270009, 13)


In [100]:
dfr

,PERMNO,date,TICKER,CUSIP,DIVAMT,PRC,VOL,RET,12-mo_return,return_std,Share_Turnover,Skewness,Sharpe_Ratio,yr_mo,log_turnover
11,10026,20101231,JJSF,46603210,0.1175,48.24000,11440.0,0.056995,0.164315,2.612977,150588.0,49.690445,0.062884,201012,11.922303
12,10026,20110131,JJSF,46603210,0.0000,42.47000,12928.0,-0.11961,0.004683,2.588286,145657.0,50.228218,0.001809,201101,11.889010
13,10026,20110228,JJSF,46603210,0.0000,43.97000,13651.0,0.035319,0.021624,2.586358,148426.0,50.387447,0.008361,201102,11.907842
14,10026,20110331,JJSF,46603210,0.1175,47.07000,14227.0,0.073175,0.019961,2.774392,148809.0,47.296852,0.007195,201103,11.910419
15,10026,20110429,JJSF,46603210,0.0000,50.82000,8807.0,0.079669,0.153950,3.327660,141075.0,39.750910,0.046264,201104,11.857047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309543,93436,20200831,TSLA,88160R10,0.0000,498.32001,4051970.0,0.741452,1.068834,331.770356,37269874.0,5.856754,0.003222,202008,17.433696
309544,93436,20200930,TSLA,88160R10,0.0000,429.01001,17331954.0,-0.139087,0.362282,316.257290,53235913.0,6.292764,0.001146,202009,17.790244
309545,93436,20201030,TSLA,88160R10,0.0000,388.04001,8330610.0,-0.095499,0.176093,310.129898,59215398.0,6.476036,0.000568,202010,17.896692
309546,93436,20201130,TSLA,88160R10,0.0000,567.59998,7811501.0,0.462736,0.356824,295.036194,65448048.0,7.008725,0.001209,202011,17.996767


Litigation:

In [101]:
dfl = pd.read_csv('../Data/Litigation_Data/Corporate_Lit_Outcomes_AggByYear.csv')
dfl

,Ticker,Year,Securities,Total_Cases,CIK Code
0,A,2001,1.0,1.0,1090872.0
1,A,2002,0.0,1.0,1090872.0
2,A,2003,0.0,2.0,1090872.0
3,A,2004,0.0,1.0,1090872.0
4,A,2005,0.0,2.0,1090872.0
...,...,...,...,...,...
67931,ZYXI,2017,0.0,0.0,846475.0
67932,ZYXI,2020,0.0,0.0,846475.0
67933,ZYXI,2016,0.0,0.0,846475.0
67934,ZYXI,2019,0.0,0.0,846475.0


Merging:

In [107]:
df = dff[['yr_mo', 'fyear', 'tic', 'conm', 'cik', 'ggroup', 'gind', 'gsector', 'gsubind', 'log_assets', 'sales_growth']] # russell 3000 info
print(df.shape)

df = df.merge(dfr[['yr_mo', 'TICKER','return_std', 'log_turnover', 'Skewness', 'Sharpe_Ratio']], 
        left_on = ['yr_mo', 'tic'], right_on = ['yr_mo', 'TICKER'], how = 'left')            # return info
df = df.dropna()
print(df.shape)

df = df.merge(dfl.iloc[:,:4], how = 'left', left_on = ['tic', 'fyear'], 
              right_on = ['Ticker', 'Year']).drop(columns = ['Ticker', 'TICKER', 'Year']).fillna(0)  # litigation info

(25078, 11)
(20487, 16)


In [111]:
df['securities_binary'] = np.where(df['Securities'] > 0, 1, 0)
df['litigation_binary'] = np.where(df['Total_Cases'] > 0, 1, 0)
df

,yr_mo,fyear,tic,conm,cik,ggroup,gind,gsector,gsubind,log_assets,sales_growth,return_std,log_turnover,Skewness,Sharpe_Ratio,Securities,Total_Cases,securities_binary,litigation_binary
0,201105,2010.0,AIR,AAR CORP,1750.0,2010,201010,20,20101010,7.440573,0.313302,4.600250,13.596923,14.991578,0.126285,0.0,0.0,0,0
1,201205,2011.0,AIR,AAR CORP,1750.0,2010,201010,20,20101010,7.694235,0.168217,4.627977,13.675111,13.128846,-0.117570,0.0,0.0,0,0
2,201305,2012.0,AIR,AAR CORP,1750.0,2010,201010,20,20101010,7.667111,0.044638,2.023224,13.634210,24.818062,0.252264,0.0,0.0,0,0
3,201405,2013.0,AIR,AAR CORP,1750.0,2010,201010,20,20101010,7.695985,-0.060957,2.444836,13.549953,32.604433,0.048756,0.0,0.0,0,0
4,201505,2014.0,AIR,AAR CORP,1750.0,2010,201010,20,20101010,7.323171,-0.216560,1.852432,13.391156,45.178438,0.044659,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20482,202012,2020.0,ATH,ATHENE HOLDING LTD,1527469.0,4030,403010,40,40301020,12.219833,-0.091893,6.485948,15.348588,16.156851,-0.001487,0.0,0.0,0,0
20483,201912,2019.0,NVT,NVENT ELECTRIC PLC,1720635.0,2010,201040,20,20104010,8.442534,-0.004337,2.170105,14.931409,34.067477,0.023206,0.0,0.0,0,0
20484,202012,2020.0,NVT,NVENT ELECTRIC PLC,1720635.0,2010,201040,20,20104010,8.381625,-0.093194,2.726119,14.866853,22.082679,-0.013406,0.0,0.0,0,0
20485,201912,2019.0,ACA,ARCOSA INC,1739445.0,2010,201030,20,20103010,7.741751,0.189332,4.177775,13.747747,25.282000,0.124602,0.0,1.0,0,1


In [116]:
df.to_csv('Russell3000Regression.csv', index = False)